<a href="https://colab.research.google.com/github/Krishna4123/amdl/blob/main/EngTOTamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
english_sentences = [
    "i love apples",
    "you like bananas",
    "she eats bread",
    "he drinks water",
    "we read books",
    "they play football",
    "i am happy",
    "you are my friend",
    "she is a teacher",
    "he is a student",
    "we go to school",
    "they watch movies",
    "i eat rice",
    "you drink milk",
    "she likes flowers",
    "he loves music",
    "we speak english",
    "they learn tamil",
    "i see the sun",
    "you see the moon"
]

tamil_sentences = [
    "நான் ஆப்பிள்களை விரும்புகிறேன்",
    "நீ வாழைப்பழங்களை விரும்புகிறாய்",
    "அவள் ரொட்டி சாப்பிடுகிறாள்",
    "அவன் தண்ணீர் குடிக்கிறான்",
    "நாம் புத்தகங்கள் படிக்கிறோம்",
    "அவர்கள் கால்பந்து விளையாடுகிறார்கள்",
    "நான் மகிழ்ச்சியாக இருக்கிறேன்",
    "நீ என் நண்பன்",
    "அவள் ஒரு ஆசிரியர்",
    "அவன் ஒரு மாணவன்",
    "நாம் பள்ளிக்கு செல்கிறோம்",
    "அவர்கள் திரைப்படங்களை பார்க்கிறார்கள்",
    "நான் சாதம் சாப்பிடுகிறேன்",
    "நீ பால் குடிக்கிறாய்",
    "அவள் மலர்களை விரும்புகிறாள்",
    "அவன் இசையை விரும்புகிறான்",
    "நாம் ஆங்கிலம் பேசுகிறோம்",
    "அவர்கள் தமிழ் கற்கிறார்கள்",
    "நான் சூரியனை பார்க்கிறேன்",
    "நீ சந்திரனை பார்க்கிறாய்"
]


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dot, Activation, Concatenate
from tensorflow.keras.models import Model
import numpy as np


In [3]:
# Source (English)
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(english_sentences)
src_sequences = src_tokenizer.texts_to_sequences(english_sentences)
src_vocab_size = len(src_tokenizer.word_index) + 1

# Target (Tamil)
tgt_tokenizer = Tokenizer()
tgt_tokenizer.fit_on_texts(tamil_sentences)
tgt_sequences = tgt_tokenizer.texts_to_sequences(tamil_sentences)
tgt_vocab_size = len(tgt_tokenizer.word_index) + 1

max_src_len = max(len(s) for s in src_sequences)
max_tgt_len = max(len(s) for s in tgt_sequences)

src_padded = pad_sequences(src_sequences, maxlen=max_src_len, padding='post')
tgt_padded = pad_sequences(tgt_sequences, maxlen=max_tgt_len, padding='post')


In [4]:
embed_dim = 32
units = 64


In [5]:
encoder_inputs = Input(shape=(max_src_len,))
enc_emb = Embedding(src_vocab_size, embed_dim)(encoder_inputs)

encoder_lstm = LSTM(units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)


In [6]:
decoder_inputs = Input(shape=(max_tgt_len,))
dec_emb = Embedding(tgt_vocab_size, embed_dim)(decoder_inputs)

decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])


In [7]:
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])


In [8]:
output = Dense(tgt_vocab_size, activation='softmax')(decoder_combined_context)

model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4, 32)     │      1,568 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 3, 32)     │      1,472 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 4, 64),   │     24,832 │ embedding[0][0]   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 3, 64),   │     24,832 │ embedding_1[0][0… │
│                     │ (None, 64),       │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 3, 4)      │          0 │ lstm_1[0][0],     │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 3, 4)      │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 3, 64)     │          0 │ activation[0][0], │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3, 128)    │          0 │ dot_1[0][0],      │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 46)     │      5,934 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 58,638 (229.05 KB)

 Trainable params: 58,638 (229.05 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(
    [src_padded, tgt_padded],
    np.expand_dims(tgt_padded, -1),
    batch_size=2,
    epochs=300
)


Epoch 1/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 3.8288
Epoch 2/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.8136 
Epoch 3/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.8037 
Epoch 4/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.7861 
Epoch 5/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.7664 
Epoch 6/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.7314 
Epoch 7/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.6767 
Epoch 8/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.5899 
Epoch 9/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.3842 
Epoch 10/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.2413 
Epoch 11/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.7706 
Epoch 12/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.5141
Epoch 13/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3611 
Epoch 14/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.1013 
Epoch 15/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss:

In [10]:
def translate(sentence):
    seq = src_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_src_len, padding='post')

    decoder_input = np.zeros((1, max_tgt_len))
    output = model.predict([seq, decoder_input])

    out_seq = np.argmax(output[0], axis=-1)

    inv_map = {v: k for k, v in tgt_tokenizer.word_index.items()}
    words = [inv_map.get(i, '') for i in out_seq if i > 0]

    return " ".join(words)


In [11]:
print("English: i love apples")
print("Tamil:", translate("i love apples"))

print("\nEnglish: you drink milk")
print("Tamil:", translate("you drink milk"))

print("\nEnglish: they learn tamil")
print("Tamil:", translate("they learn tamil"))


English: i love apples
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
Tamil: நான் ஆப்பிள்களை விரும்புகிறேன்

English: you drink milk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Tamil: நீ பால் குடிக்கிறாய்

English: they learn tamil
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Tamil: அவர்கள் தமிழ் கற்கிறார்கள்
